In [284]:
# Needed modules :
import os
import gc
import copy
import sklearn

import Drive

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt


from sys import getsizeof
from typing import List

import Preprocessing as pre
from functools import reduce

In [265]:
# Function definitions :

def time_indexed_df(df1: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    _tmp = copy.deepcopy(df1)
    _tmp.index = df1.dateTime
    _tmp.drop('dateTime', axis=1, inplace=True)
    _tmp = _tmp.sort_index()
    gc.collect()
    return _tmp

def fill_nas(df1: pd.core.frame.DataFrame, 
             col_names: List[str] = ['activeInsulin', 'carbs', 'insulin', 'trend']) -> pd.core.frame.DataFrame:
    _tmp = copy.deepcopy(df1)
    for name in col_names:
        _tmp[name] = df1[name].fillna(0)
    gc.collect()
    return _tmp


In [266]:
d = Drive.Drive()

In [267]:
file_name = 'journal.jl'
file_path = os.path.join('data', file_name)
d.download(file_name   = file_name, 
           target_name = file_path
          )

True

In [268]:
pre.file_filter(file_path)

In [269]:
ls data/

journal_filtered.jl  journal.jl


In [270]:
_raw = pd.read_json(file_path, lines=True)
_raw.head()

,BG,IG,LOT,REF,activeInsulin,carbs,dateTime,details,food,initSuccess,insulin,secondRound,trend,type
0,227.0,226.0,NaN,NaN,NaN,0.0,2019-02-07 00:27:00,Normal,None,NaN,1.05,NaN,0.0,data
1,NaN,190.0,NaN,NaN,NaN,NaN,2019-02-07 01:26:00,Normal,None,NaN,NaN,NaN,NaN,data
2,NaN,174.0,NaN,NaN,NaN,NaN,2019-02-07 08:49:00,Wake,None,NaN,NaN,NaN,0.0,event
3,NaN,102.0,NaN,NaN,NaN,NaN,2019-02-07 05:04:00,Suspension before hypo,None,NaN,NaN,NaN,0.0,alarm
4,NaN,98.0,NaN,NaN,NaN,NaN,2019-02-07 06:25:00,Basal resumption,None,NaN,NaN,NaN,0.0,alarm


In [271]:
#_raw[_raw.index.duplicated()]

In [304]:
undesired_columns = [ 
    'LOT',
    'REF', 
    'initSuccess', 
    'secondRound',
    'food'
]

In [305]:
_tmp = (
        _raw.drop(undesired_columns, axis=1)
       ).loc[
                (_raw['type'] == 'data') |
                (_raw['type'] == 'event')
            ]

In [306]:
_t_data = time_indexed_df(_tmp)
_tmp.head()

,BG,IG,activeInsulin,carbs,dateTime,details,insulin,trend,type
0,227.0,226.0,NaN,0.0,2019-02-07 00:27:00,Normal,1.05,0.0,data
1,NaN,190.0,NaN,NaN,2019-02-07 01:26:00,Normal,NaN,NaN,data
2,NaN,174.0,NaN,NaN,2019-02-07 08:49:00,Wake,NaN,0.0,event
5,NaN,148.0,NaN,NaN,2019-02-07 02:24:00,Normal,NaN,NaN,data
6,NaN,131.0,NaN,NaN,2019-02-07 03:14:00,Normal,NaN,NaN,data


In [307]:
_t_data.head()

,BG,IG,activeInsulin,carbs,details,insulin,trend,type
dateTime,,,,,,,,
2019-02-07 00:27:00,227.0,226.0,NaN,0.0,Normal,1.05,0.0,data
2019-02-07 01:26:00,NaN,190.0,NaN,NaN,Normal,NaN,NaN,data
2019-02-07 02:24:00,NaN,148.0,NaN,NaN,Normal,NaN,NaN,data
2019-02-07 03:14:00,NaN,131.0,NaN,NaN,Normal,NaN,NaN,data
2019-02-07 04:24:00,NaN,118.0,NaN,NaN,Normal,NaN,NaN,data


In [308]:
data = fill_nas(_t_data)
data.head()

,BG,IG,activeInsulin,carbs,details,insulin,trend,type
dateTime,,,,,,,,
2019-02-07 00:27:00,227.0,226.0,0.0,0.0,Normal,1.05,0.0,data
2019-02-07 01:26:00,NaN,190.0,0.0,0.0,Normal,0.00,0.0,data
2019-02-07 02:24:00,NaN,148.0,0.0,0.0,Normal,0.00,0.0,data
2019-02-07 03:14:00,NaN,131.0,0.0,0.0,Normal,0.00,0.0,data
2019-02-07 04:24:00,NaN,118.0,0.0,0.0,Normal,0.00,0.0,data


In [332]:
data2 = copy.deepcopy(data)

In [375]:
data2['BG'] = list( 
                    map(
                         lambda x, y: x if not np.isnan(x) else y, data['IG'], data['BG']
                       )
                  )                 

In [376]:
data2['IG'] = list( 
                    map(
                         lambda x, y: x if not np.isnan(x) else y, data['IG'], data['BG']
                       )
                  )  

In [377]:
data2.head()

,BG,IG,activeInsulin,carbs,details,insulin,trend,type
dateTime,,,,,,,,
2019-02-07 00:27:00,226.0,226.0,0.0,0.0,Normal,1.05,0.0,data
2019-02-07 01:26:00,190.0,190.0,0.0,0.0,Normal,0.00,0.0,data
2019-02-07 02:24:00,148.0,148.0,0.0,0.0,Normal,0.00,0.0,data
2019-02-07 03:14:00,131.0,131.0,0.0,0.0,Normal,0.00,0.0,data
2019-02-07 04:24:00,118.0,118.0,0.0,0.0,Normal,0.00,0.0,data


In [383]:
data2.isna().sum()

BG               2
IG               2
activeInsulin    0
carbs            0
details          0
insulin          0
trend            0
type             0
dtype: int64

In [386]:
data3 = data2.dropna()

In [390]:
data3

,BG,IG,activeInsulin,carbs,details,insulin,trend,type
dateTime,,,,,,,,
2019-02-07 00:27:00,226.0,226.0,0.00,0.0,Normal,1.05,0.0,data
2019-02-07 01:26:00,190.0,190.0,0.00,0.0,Normal,0.00,0.0,data
2019-02-07 02:24:00,148.0,148.0,0.00,0.0,Normal,0.00,0.0,data
2019-02-07 03:14:00,131.0,131.0,0.00,0.0,Normal,0.00,0.0,data
2019-02-07 04:24:00,118.0,118.0,0.00,0.0,Normal,0.00,0.0,data
2019-02-07 08:49:00,174.0,174.0,0.00,0.0,Wake,0.00,0.0,event
2019-02-07 09:24:00,184.0,184.0,0.00,30.0,Normal,3.75,0.0,data
2019-02-07 11:24:00,195.0,195.0,0.00,0.0,Postprandial,0.00,0.0,event
2019-02-07 12:53:00,133.0,133.0,0.00,56.0,Normal,4.95,0.0,data
